In [1]:
import numpy as np
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
import os
import time

In [2]:
meta = pd.read_csv('synthetic-meta.txt', sep=',', header=None)
meta.columns = ['station', 'direction', 'date']
z_meta = meta[meta.direction == 'Z']

In [3]:
def process_csv(station):
    df = pd.read_csv('../Artificial_data/DOGEx_v1/csv/' + station, header=None)

    df.columns = ['date', 'un', 'ue', 'uz', 'sn', 'se', 'sz']
    df.drop(['sn', 'se', 'sz'], axis=1, inplace=True)
    df = df[['date','uz']]

    offset_dates = z_meta[z_meta.station==station[0:4]]['date']
    
    return df, offset_dates

In [4]:
interval_size = 40
offset = 10

num_files = 0
tot_ranges = 0

csv_list = []
ranges_per_csv = []

for file in os.listdir('../Artificial_data/DOGEx_v1/csv/'):
    if file[0] == '.':
        continue 
    
    num_ranges = (sum(1 for line in open('../Artificial_data/DOGEx_v1/csv/' + file)) - interval_size) // offset
        
    csv_list.append(file)
    ranges_per_csv.append(num_ranges)
    
    tot_ranges += num_ranges
    num_files +=  1

In [5]:
ranges = np.empty((tot_ranges, 3), dtype=np.ndarray)

In [6]:
range_row = 0
csv_idx = 0
while range_row < tot_ranges:
    df, offset_dates = process_csv(csv_list[csv_idx])    
    i = 0
    while i < ranges_per_csv[csv_idx]:
        arr = np.array(df.iloc[i * offset:i * offset + interval_size].T)

        for date in arr[0]:
            if date in offset_dates.values:
                ranges[range_row] = (arr[0], arr[1], 1)
                break
            else:
                ranges[range_row] = (arr[0], arr[1], 0)

        i = i + 1  
        range_row = range_row + 1   
    
    csv_idx += 1

In [7]:
data = pd.DataFrame(ranges)
data.head()

,0,1,2
0,"[1992.1834, 1992.2218, 1992.2464, 1992.2847, 1...","[0.0014494, -0.02434133, -0.02490843, -0.02406...",0
1,"[1992.3395, 1992.3422, 1992.345, 1992.3614, 19...","[-0.00336805, -0.02174591, -0.00028298, -0.018...",0
2,"[1992.4326, 1992.4353, 1992.4381, 1992.4408, 1...","[0.00067229, -0.01755925, -0.03052116, 0.03938...",0
3,"[1992.4764, 1992.4791, 1992.4819, 1992.4846, 1...","[-0.03043266, -0.0040784, -0.01701879, -0.0095...",0
4,"[1992.5065, 1992.5092, 1992.512, 1992.5147, 19...","[0.00880199, 0.02723057, 0.02394497, 0.0008241...",0


In [8]:
data.columns = ['dates', 'uz', 'label']
regular = data[data.label == 0]
offsets  = data[data.label == 1]

In [9]:
offsets_upsampled = resample(offsets, replace=True, n_samples=len(regular), random_state=42)
data_upsampled = pd.concat([regular, offsets_upsampled])

In [10]:
data_upsampled.label.value_counts()

X = X = np.stack(np.array(data_upsampled['uz']))
y = np.stack(np.array(data_upsampled['label']))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
X_train[0:200, :]
y_test.shape

(16871,)

In [13]:
from sktime.classification.dictionary_based import IndividualBOSS
from sktime.datasets import load_unit_test

In [ ]:
start = time.time()

y_pred = clf.predict(X_test)

end = time.time()
print(end - start)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()